# Spatial Statistics and Geospatial data visualization

## Key concepts in Geospatial data

### Raster and vector data

Raster data is a grid of pixels, each pixel representing a value. Raster data is often used to represent continuous data, such as elevation, temperature, or precipitation. Raster data is often stored as a file with a .tif extension.

Vector data is a collection of points, lines, and polygons. Vector data is often used to represent discrete data, such as the location of a city, the boundaries of a country, or the location of a river. Vector data is often stored as a file with a .shp extension.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import rasterio as rs
import rasterio.plot

import tempfile

import numpy as np
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages

import folium
import folium.plugins

from math import log10

from scipy import misc
from PIL import Image


rpy2.robjects.numpy2ri.activate()

In [ ]:
# Then, using the language of your choice (R or Python), design a non-interactive plot of French school districts

# Read in the data in .shp format
frenchAcademyDf = gpd.read_file('academies-20160209-shp/academies-20160209.shp')
frenchAcademyDf

In [ ]:
# Keep only metropolitan France
# Plot and color the data
colors = [plt.get_cmap('viridis')(i * 20) for i in range(len(frenchAcademyDf["wikipedia"]))]
frenchAcademyDf.clip([-5, 41, 10, 51]).plot(color=colors, edgecolor='black')

In [ ]:
# Find a shapefile representing the entire world, 
# and design a set of maps based on different pro-jections / coordinate reference systems (CRS) (ex.: Mercator, etc.).

# Read in the data in .shp format
worldDf = gpd.read_file('world-administrative-boundaries/world-administrative-boundaries.shp')
worldDf.plot()

In [ ]:
# Mercator projection
worldDf.to_crs(epsg=3395).plot(cmap='viridis')

In [ ]:
# Lambert Azimuthal Equal Area projection
worldDf.to_crs(epsg=9822).plot(cmap='viridis')

In [ ]:
# Polar Stereographic projection
worldDf.to_crs(epsg=3031).plot(cmap='viridis')

Write down a paragraph (in your notebook only) explaining the concept of CRS.

CRS means Coordinate Reference System and is a framework used to precisely measure locations on the surface of the Earth as coordinates. It is thus the application of the abstract mathematics of coordinate systems and analytic geometry to geographic space.

In [ ]:
# Download a raster dataset from the SRTM website (elevation data from the entire world, +-30 meters in precision). Design a 2D map of the area you selected. The size of the area of interest mostly depends upon your laptop’s computing power, since rendering a map can be CPU intensive.

# Read in the data in .tif format
img = rs.open('output_SRTMGL1.tif')

# Plot the data
rs.plot.show(img, cmap='terrain', title='Elevation of the World', )

### Digital Elevation Models



In [ ]:
def rayshade(z, img_path=None, zscale=10, fov=0, theta=135, zoom=0.75, phi=45, windowsize=(1000, 1000)):
    
    # Output path.
    if not img_path:
        img_path = tempfile.NamedTemporaryFile(suffix='.png').name
    
    # Import needed packages.
    rayshader = rpackages.importr('rayshader')
    
    # Convert array to matrix.
    z = np.asarray(z)
    rows, cols, _ = z.shape
    z_mat = ro.r.matrix(z, nrow=rows, ncol=cols)
    ro.globalenv['elmat'] = z_mat
    
    # Save python state to r.
    ro.globalenv['img_path'] = img_path
    ro.globalenv['zscale'] = zscale
    ro.globalenv['fov'] = fov
    ro.globalenv['theta'] = theta
    ro.globalenv['zoom'] = zoom
    ro.globalenv['phi'] = phi
    ro.globalenv['windowsize'] = ro.IntVector(windowsize)
    
    # Do the render.
    ro.r('''
        elmat %>%
          sphere_shade(texture = "desert") %>%
          add_water(detect_water(elmat), color = "desert") %>%
          add_shadow(ray_shade(elmat, zscale = 3), 0.5) %>%
          add_shadow(ambient_shade(elmat), 0) %>%
          plot_3d(elmat, zscale = zscale, fov = fov, theta = theta, zoom = zoom, phi = phi, windowsize = windowsize)
        Sys.sleep(0.2)
        render_snapshot(img_path)
    ''')
    
    # Return path.
    return img_path

In [ ]:
SRTMG = plt.imread('output_SRTMGL1.tif')
img_path = rayshade(SRTMG)

from IPython.display import Image
Image(filename=img_path) 

## Interactive data visualization with Leaflet or Folium

In [ ]:
# Create an interactive choropleth of French school districts

# Create a column in frenchAcademyDf that contains the percentage of exam success generated randomly uniformly between 0 and 100
frenchAcademyDf['perc. exam success'] = np.random.randint(0, 100, len(frenchAcademyDf))

frenchAcademyDf.explore(column='perc. exam success', cmap='viridis')


In [ ]:
# importing new datasets

phdv3 = pd.read_csv('PhD.v3.csv', sep=',', index_col=0)

phdv3["Date de soutenance"] = pd.to_datetime(phdv3["Date de soutenance"], format='%d-%m-%y')

opendata = pd.read_csv('opendata.csv', sep=';')
opendata["etablissement_rec"] = opendata["Libellé"].astype(str)

phdOpendata = pd.merge(phdv3, opendata, on='etablissement_rec', how='inner')
phdOpendata["Latitude"] = phdOpendata["Géolocalisation"].str.split(",", expand=True)[0].astype(float)
phdOpendata["Longitude"] = phdOpendata["Géolocalisation"].str.split(",", expand=True)[1].astype(float)
phdOpendata

In [ ]:
phdGeoCounts = pd.DataFrame(phdOpendata[["Latitude", "Longitude", "Date de soutenance", "etablissement_rec"]].value_counts())
phdGeoCounts = phdGeoCounts.reset_index()
phdGeoCounts.columns = ["Latitude", "Longitude", "Date de soutenance","Etablissement", "Count"]
phdGeoCounts["Etablissement"] = phdGeoCounts["Etablissement"].astype(str)
phdGeoCounts["Count"] = phdGeoCounts["Count"].astype(float)
phdGeoCounts

In [ ]:
# bubble map of number of thesis per establishment in phdOpendata

m = folium.Map(location=[48.8566, 2.3522], zoom_start=6)

for i in range(0,len(phdGeoCounts)):
    folium.Circle(
        location=[phdGeoCounts.iloc[i]['Latitude'], phdGeoCounts.iloc[i]['Longitude']],
        popup=phdGeoCounts.iloc[i]['Etablissement'],
        radius=phdGeoCounts.iloc[i]['Count'],
        color='crimson',
        fill=True,
        fill_color='crimson'
    ).add_to(m)

# Add marker on Cy tech
folium.Marker([49.0351, 2.0697], popup='CY Tech').add_to(m)

# Add a time slider depending on "Date de soutenance"
# m.add_child(folium.plugins.TimestampedGeoJson(
#     {'type': 'FeatureCollection',
#         'features': [{'type': 'Feature',
#             'geometry': {'type': 'Point',
#                 'coordinates': [phdGeoCounts.iloc[i]['Longitude'], phdGeoCounts.iloc[i]['Latitude']]
#                 },
#             'properties': {'time': phdGeoCounts.iloc[i]['Date de soutenance'].strftime('%Y/%m/%d'),
#                 'style': {'color': 'crimson'},
#                 'icon': 'circle',
#                 'iconstyle': {
#                     'fillColor': 'crimson',
#                     'fillOpacity': 0.5,
#                     'stroke': 'false',
#                     'radius': log10(phdGeoCounts.iloc[i]['Count'])
#                     }
#             }
#         }
#     for i in range(0,len(phdGeoCounts))]
#     },
#     period='P1M',
#     add_last_point=True,
#     auto_play=True,
#     loop=True,
#     max_speed=1,
#     loop_button=True,
#     date_options='YYYY/MM/DD',
#     time_slider_drag_update=True))


m

## 5 Group challenges

### 5.1 Space-time trajectory

In [99]:
whiteStorkDf = pd.read_csv('MPIAB White Stork South African Population Argos.csv', sep=',', parse_dates=['timestamp'])
whiteStorkDf["timecontinuous"] = whiteStorkDf["timestamp"].astype(int)
whiteStorkDf = gpd.GeoDataFrame(whiteStorkDf, geometry=gpd.points_from_xy(whiteStorkDf["location-long"], whiteStorkDf["location-lat"]))
whiteStorkDf.sort_values(by=['timestamp'], inplace=True)
whiteStorkDf

,event-id,visible,timestamp,location-long,location-lat,algorithm-marked-outlier,argos:altitude,argos:best-level,argos:calcul-freq,argos:iq,...,argos:sensor-4,argos:valid-location-algorithm,manually-marked-outlier,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name,timecontinuous,geometry
2193,15391850,True,2000-11-23 14:01:39,18.556,-33.901,NaN,0.0,-129.0,401.653011,54,...,2,1,NaN,argos-doppler-shift,Ciconia ciconia,20526,20526C,MPIAB White Stork South African Population Argos,974988099000000000,POINT (18.55600 -33.90100)
2194,15391851,True,2000-11-23 14:53:19,18.563,-33.880,NaN,0.0,-126.0,401.652896,66,...,2,1,NaN,argos-doppler-shift,Ciconia ciconia,20526,20526C,MPIAB White Stork South African Population Argos,974991199000000000,POINT (18.56300 -33.88000)
2195,15391852,True,2000-11-23 15:04:22,18.589,-33.853,NaN,0.0,-121.0,401.652887,56,...,1,1,NaN,argos-doppler-shift,Ciconia ciconia,20526,20526C,MPIAB White Stork South African Population Argos,974991862000000000,POINT (18.58900 -33.85300)
2196,15391853,True,2000-11-23 16:31:54,18.542,-33.870,NaN,0.0,-124.0,401.652768,55,...,2,1,NaN,argos-doppler-shift,Ciconia ciconia,20526,20526C,MPIAB White Stork South African Population Argos,974997114000000000,POINT (18.54200 -33.87000)
2197,15391854,True,2000-11-23 16:44:54,18.524,-33.874,NaN,0.0,-128.0,401.652767,6,...,2,1,NaN,argos-doppler-shift,Ciconia ciconia,20526,20526C,MPIAB White Stork South African Population Argos,974997894000000000,POINT (18.52400 -33.87400)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,15388763,True,2002-10-21 08:46:33,34.722,-2.353,NaN,0.0,-131.0,401.652620,50,...,164,1,NaN,argos-doppler-shift,Ciconia ciconia,20527,20527D,MPIAB White Stork South African Population Argos,1035189993000000000,POINT (34.72200 -2.35300)
4989,15387567,True,2002-10-21 11:07:50,27.180,11.332,NaN,0.0,-130.0,401.652027,50,...,118,1,NaN,argos-doppler-shift,Ciconia ciconia,20523,20523G,MPIAB White Stork South African Population Argos,1035198470000000000,POINT (27.18000 11.33200)
994,15388766,True,2002-10-22 10:51:20,34.777,-2.328,NaN,0.0,-131.0,401.651918,0,...,88,2,NaN,argos-doppler-shift,Ciconia ciconia,20527,20527D,MPIAB White Stork South African Population Argos,1035283880000000000,POINT (34.77700 -2.32800)
2191,15390643,True,2003-02-25 12:31:38,27.735,-18.232,NaN,0.0,-134.0,401.652545,60,...,52,1,NaN,argos-doppler-shift,Ciconia ciconia,20525,20525E,MPIAB White Stork South African Population Argos,1046176298000000000,POINT (27.73500 -18.23200)


In [ ]:
# Get min-max of latitude and longitude
minLat = whiteStorkDf["location-lat"].min()
maxLat = whiteStorkDf["location-lat"].max()
minLong = whiteStorkDf["location-long"].min()
maxLong = whiteStorkDf["location-long"].max()

# print them
print("Latitude: ", minLat, " - ", maxLat)
print("Longitude: ", minLong, " - ", maxLong)

# Create a map and export it to png
m = folium.Map(location=[(minLat + maxLat)/2, (minLong + maxLong)/2], zoom_start=6)
m.save('map.html')

In [132]:
# Make a space-time trajcetory cube of the white stork data

fig = px.scatter_3d(whiteStorkDf, x="location-long", y="location-lat", z="timestamp", opacity=0.5, color="argos:altitude")

fig.update_layout(scene = dict(
                    xaxis_title='Longitude',
                    yaxis_title='Latitude',
                    zaxis_title='Time'))
fig.update_coloraxes(showscale=False)



fig.show()

In [76]:
# Make a space-time trajcetory cube of the white stork data

fig = px.line_3d(whiteStorkDf, x="location-long", y="location-lat", z="timestamp")
fig.update_traces(line=dict(width=5))

im = Image.open('map.png')

im_x, im_y = im.size
eight_bit_img = im.convert('P', palette='WEB', dither=None)

idx_to_color = np.array(eight_bit_img.getpalette()).reshape((-1, 3))

colorscale=[[i/255.0, "rgb({}, {}, {})".format(*rgb)] for i, rgb in enumerate(idx_to_color)]


x = np.linspace(0,im_x, im_x)
y = np.linspace(0, im_y, im_y)
z = np.zeros(im.size)

fig.add_trace(go.Surface(x=x, y=y, z=z,
    surfacecolor=eight_bit_img, 
    cmin=0, 
    cmax=255,
    colorscale=colorscale,
    showscale=False,
    lighting_diffuse=1,
    lighting_ambient=1,
    lighting_fresnel=1,
    lighting_roughness=1,
    lighting_specular=0.5,

))

fig.write_html("stork.html")

In [ ]:
# Make a space-time trajcetory cube of the white stork data with a map under the trajectories

fig = px.line_3d(whiteStorkDf, x="location-long", y="location-lat", z="timestamp")
fig.update_traces(line=dict(width=5))

